# Collect satellite images from sentinel hub

#### Imports

In [ ]:
import matplotlib.pyplot as plt
from geopy.distance import geodesic
from sentinelhub import SHConfig,CRS, BBox, DataCollection, MimeType, WcsRequest, WmsRequest 
from shapely.geometry import Polygon, Point
import base64
import os
import io
from PIL import Image

#### Function to calculate longitude and latitude

Angle : The orientation angle or direction is determined

Distan : distance to the coordinates of the point to be found

In [ ]:
def find_point(angle, distan, longitude,latitude):
    point = geodesic(kilometers=distan).destination((latitude,longitude), angle)
    latitude_new = point.latitude
    longitude_new = point.longitude
    return longitude_new,latitude_new

#### Class hash table

Use longitude to hash into keys. 

In [ ]:
class hash_table:
    def __init__(self,size):
        self.table = [[]]*size
    def hash_func(self,longitude):
        longitude = int(str(longitude).replace('.',''))
        key_1 = longitude % len(self.table)
        key_2 = (longitude + key_1)% len(self.table)
        return key_1,key_2
    def set_hash(self,longitude,latitude):
        key = ''
        key_1,key_2 = self.hash_func(longitude)
        if self.table[key_1] == []:
            self.table[key_1] = [key_1,[[longitude,latitude]]]
            key = key_1
        elif self.table[key_2] == []:
            self.table[key_2] = [key_2,[[longitude,latitude]]] 
            key = key_2      
        else:
            for i in range(len(self.table)):
                if self.table[i] == []:
                    self.table[i] = [i,[[longitude,latitude]]]
                    key = i
                    break
        return key
    def get_value(self,longitude,latitude):
        key_1,key_2 = self.hash_func(longitude)
        if self.table[key_1][1][0][1] == latitude:
            return self.table[key_1]
        elif self.table[key_2][1][0][1] == latitude:
            return self.table[key_2]
        else:
            for i in range(len(self.table)):
                if self.table[i][1][0][1] == latitude:
                    return self.table[i]
    def del_value(self,key):
        self.table[key] = []
        return  

#### Class Graph

Use graph to connect all points in the map to improve the speed of finding points around a point by O(1)

In [ ]:
class Graph:
    def __init__(self):
        self.nodes = {}
    def add_node(self, key,longitude,latitude):
        if key not in self.nodes:
            self.nodes[key] = [[longitude,latitude],[]]
    def add_edge(self, key_1, key_2):
        if key_1 in self.nodes and key_2 in self.nodes:
            self.nodes[key_1][1].append(key_2)
            self.nodes[key_2][1].append(key_1)            
    def get_graph(self):
        return self.nodes
    def neighbors(self, key):
        if key in self.nodes:
            return self.nodes[key][1]
        else:
            return []

#### Delineate the area where images need to be collected

Visit the website to localize the area where data is needed

#https://apps.sentinel-hub.com/requests-builder/ 

In [ ]:
graph = Graph()
hash = hash_table(10000)
selected_area = [
      [12.44693,41.8941],
      [12.484271,41.930379],
      [12.484271,41.930379],
      [12.565332,41.895888],
      [12.536145,41.844501],
      [12.423519,41.84399],
      [12.399483,41.880042],
      [12.44693,41.8941]
    ]
polygon = Polygon(selected_area)

#### Determine the number of rows and columns to scan through points on the map.

In [ ]:
number_of_column = 25
number_of_row = 22

list_of_connection_points = [[[]]*(number_of_column+1) for i in range(0,2)]

#List of longitudes corresponding to each latitude. This list is used to find any point on the map.
list_longitudes_align_with_latitude = {'latitude': []}

#Location of the starting point to search for points in the area to collect data
starting_point = [12.328209,41.786263]

#Distance between points
distance = 1
distance_increases = 0.01

#Points outside the area need to collect data
count = 0


#### The algorithm searches for points within the data collection area

In [ ]:
#Loop to find the first column points
for i in range(0,number_of_row):    
    
    point_beginning_row = find_point(0,i*distance, starting_point[0], starting_point[1])
    
    point_to_check = Point(point_beginning_row[0],  point_beginning_row[1])
    
    #Check if the point belongs to the area where data needs to be collected. If so, use a hash function to get it into the table
    if polygon.contains(point_to_check):
        key_1 = hash.set_hash(starting_point[0],point_beginning_row[1])
          
    list_longitudes_align_with_latitude['latitude'].append(point_beginning_row[1])
    
    list_longitudes_align_with_latitude[point_beginning_row[1]] = []
    
    #Iterate to find each point in the row
    for j in range(0,number_of_column):
        points_on_the_row = find_point(90,j*distance , point_beginning_row[0] , point_beginning_row[1])
        points_on_the_row_45 = find_point(45,j*distance_increases , points_on_the_row[0] , points_on_the_row[1])
        
        point_to_check = Point(points_on_the_row[0],point_beginning_row[1])
        
        #Check if the point belongs to the area where data needs to be collected. If so, use a hash function to get it into the table
        if polygon.contains(point_to_check):
            key_1 = hash.set_hash(points_on_the_row[0],point_beginning_row[1])
            
            #Add points to the graph
            graph.add_node(key_1,points_on_the_row_45[0],points_on_the_row_45[1])
 
            list_of_connection_points[1][j] = key_1
            
            #Create connections between points in the map
            list_try = [j-1,j,j+1]
            for z in list_try:
                try:
                    graph.add_edge(key_1,list_of_connection_points[0][z])
                except:
                    pass
            if j == 0:
                key_2 = key_1
            else:
                graph.add_edge(key_1,key_2)
                key_2 = key_1   
            list_longitudes_align_with_latitude[point_beginning_row[1]].append(points_on_the_row[0])
        #If the point is not in the area where data needs to be collected, only put the point in the graph and not in the hash table.
        else:
            #Add points to the graph
            graph.add_node(f'None{count}',points_on_the_row_45[0],points_on_the_row_45[1])
            
            list_of_connection_points[1][j] = f'None{count}'
            
            #Create connections between points in the map
            list_try = [j-1,j,j+1]
            for z in list_try:
                try:
                    graph.add_edge(f'None{count}',list_of_connection_points[0][z])
                except:
                    pass
            if j == 0:
                key_2 = f'None{count}'
            else:
                graph.add_edge(f'None{count}',key_2)
                key_2 = f'None{count}'  
        count +=1            
        
    list_of_connection_points[0] = list_of_connection_points[1]
    list_of_connection_points[1] = [[]]*number_of_column

#### Use self.neighbor of the graph to put information about 8 points around 1 point into the hash table

In [ ]:

for i in graph.nodes:
    #Check for points that do not start from None
    if not str(i).startswith('None'):
        neigh = graph.neighbors(i)
        
        hash.table[i][1].append(graph.nodes[neigh[-1]][0])
        hash.table[i].append(neigh)
        
        plt.scatter(hash.table[i][1][0][0],hash.table[i][1][0][1],marker='x')
        plt.annotate(f"{i}",(hash.table[i][1][0][0],hash.table[i][1][0][1]), textcoords="offset points", xytext=(0,10), ha='center')    
  
latitudes = [coord[1] for coord in selected_area]
longitudes = [coord[0] for coord in selected_area]
plt.plot(longitudes, latitudes, marker='o', linestyle='-')

plt.ylabel('Latitude')
plt.xlabel('Longitude')
plt.title('Points located in the area where data needs to be collected')

plt.grid(True)
plt.show() 

#### Use binary search to search for any point in any area in the area that needs to collect data

In [ ]:
def binary_search(list_search, target):
    left, right = 0, len(list_search) - 1
    prev_index = ''
    
    for i in range(len(list_search)):
        mid = left + (right - left) // 2
        
        if list_search[mid] > target:
   
            right = mid - 1
        else:
            prev_index = mid  
            left = mid + 1
    return list_search[prev_index]

#### Example of finding random points in the area where data needs to be collected

In [ ]:
point_search = [12.4887,41.9004]
try:
    latitude = binary_search(list_longitudes_align_with_latitude['latitude'],point_search[1])
    longitude = binary_search(list_longitudes_align_with_latitude[latitude],point_search[0])
    print(hash.get_value(longitude,latitude))
except:
    print('not in hash.table')

#### Function used to draw a magnetic image after collecting data

In [ ]:
def plot_image(image):
    plt.title('sentinal')
    plt.ylabel('Latitude')
    plt.xlabel('Longitude')
    plt.imshow(image)

#### Connect your mysql server

In [ ]:
import mysql.connector

host = "127.0.0.1"
user = "root"
password = "*********"
 
conn = mysql.connector.connect(
    host=host,
    user=user,
    password=password,
)

if conn.is_connected():
    print("connected")

cur = conn.cursor()

#### Create a database to store data collected from seninel_hub

In [ ]:
cur.execute("CREATE DATABASE SentinelHubData")
cur.execute("USE SentinelHubData")

#### Create Table

In [ ]:
create_table = """
    CREATE TABLE image (
    Key_image INT PRIMARY KEY,
    Longitude float,
    Latitude float,
    Neighbors varchar(255)
);"""
cur.execute(create_table)

create_table_2 = """
    CREATE TABLE date_image (
    Key_image INT,
    Date_time datetime,
    image longblob,
    FOREIGN KEY (Key_image) REFERENCES image(key_image)
);
"""
cur.execute(create_table_2)


#### Establish connection to sentinal_hub

In [ ]:
id = '7448de78-8841-4e14-b7eb-4e68bbb2ec1d'
config = SHConfig()
config.instance_id = id

if config.instance_id == '':
    print('Unable to connect')
else:
    print('Connected successfully')

#### List of error points when collecting data

In [ ]:
list_error = []

#### Use loop to collect images from seninal_hub

In [ ]:
for i in graph.nodes:
    if not str(i).startswith('None'):
        betsiboka_coords_wgs84 = (hash.table[i][1][0][0], hash.table[i][1][0][1],hash.table[i][1][1][0],hash.table[i][1][1][1])
        print(hash.table[i][0])
        try:
            betsiboka_bbox = BBox(bbox=betsiboka_coords_wgs84, crs=CRS.WGS84)
            wms_true_color_request = WmsRequest(
                data_collection=DataCollection.SENTINEL2_L1C,
                data_folder=f"E:/data/test/{i}",
                layer="TRUE-COLOR",
                bbox=betsiboka_bbox,
                time=('2022-05-01','2022-05-31'),
                width=500,
                maxcc=0.5,
                image_format = MimeType.PNG,
                # time_difference=datetime.timedelta(hours=2),
                config=config,
            )
            wms_true_color_img = wms_true_color_request.get_data()
            # plot_image(wms_true_color_img[-1])
            wms_true_color_request.save_data()

            neighbor = [str(item) for item in hash.table[i][2]]
            neighbor = ', '.join(neighbor)
            
            image = [int(i),float(hash.table[i][1][0][0]),float(hash.table[i][1][0][0]),neighbor]
            insert_image_query = "INSERT INTO image (Key_image, Longitude, Latitude, Neighbors) VALUES (%s, %s, %s, %s)"
            
            cur.execute(insert_image_query, image)
            
            date_image = []
            for index, date in enumerate(wms_true_color_request.get_dates()):
                date_image.append(str(date))
            path = f'E:/data/test/{i}/'
            contents = os.listdir(path)
            
            for z in range(len(contents)):
                with open(f'E:/data/test/{i}/{contents[z]}/response.png','rb') as image_file:
                    base64_bytes = base64.b64encode(image_file.read())
                    
                image_date = [int(i),date_image[z],base64_bytes]
                insert_image_date_query = "INSERT INTO date_image (Key_image, Date_Time,image) VALUES (%s, %s,%s)"

                cur.execute(insert_image_date_query, image_date)
        except:
            list_error.append(i)
conn.commit()

#### Get images from mysql to your computer

In [ ]:
#blob -> png
cur.execute('select * from date_image;')
rows = cur.fetchall()

for i in rows:
    base64_string = i[-1].decode()

    img = Image.open(
        io.BytesIO(
            base64.decodebytes(bytes(base64_string, 'utf-8'))
        )
    )
    img.save(f'E:/data/sql_image/image_{i[0]}.png')
